In [2]:
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pydicom

J = os.path.join

root = J(os.environ["DATASET_ROOT"], "bcd2022")

In [4]:
df = pd.read_csv(J(root,"train.csv"))
df

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54701,1,9973,1729524723,R,MLO,43.0,0,0,0,1.0,0,C,49,False
54702,1,9989,63473691,L,MLO,60.0,0,0,0,NaN,0,C,216,False
54703,1,9989,1078943060,L,CC,60.0,0,0,0,NaN,0,C,216,False
54704,1,9989,398038886,R,MLO,60.0,0,0,0,0.0,0,C,216,True


In [14]:
p1 = df[df.cancer==1]["patient_id"]
p2 = df[df.cancer==0]["patient_id"]

In [29]:
print(len(p1), len(p2), len(set(p1.values).intersection(set(p2.values))))

1158 53548 480


In [32]:
df[df["difficult_negative_case"]==True]

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True
5,2,10011,270344397,L,MLO,55.0,0,0,0,0.0,0,NaN,21,True
28,1,10049,94335194,R,MLO,52.0,0,0,0,0.0,0,C,49,True
29,1,10049,1207499426,R,MLO,52.0,0,0,0,0.0,0,C,49,True
30,1,10049,1351270472,R,CC,52.0,0,0,0,0.0,0,C,49,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54672,2,9953,1640274067,R,MLO,60.0,0,0,0,0.0,0,NaN,48,True
54673,2,9954,21064441,L,MLO,48.0,0,0,0,0.0,0,NaN,48,True
54674,2,9954,2043422318,L,CC,48.0,0,0,0,0.0,0,NaN,48,True
54704,1,9989,398038886,R,MLO,60.0,0,0,0,0.0,0,C,216,True


In [33]:
df[df["patient_id"]==5]

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
37282,2,5,640805896,L,MLO,68.0,0,0,0,0.0,0,NaN,21,True
37283,2,5,1351088028,L,CC,68.0,0,0,0,0.0,0,NaN,21,True
37284,2,5,940388076,R,CC,68.0,0,0,0,NaN,0,NaN,21,False
37285,2,5,1633417959,R,MLO,68.0,0,0,0,NaN,0,NaN,21,False
